In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
%load_ext autoreload
%autoreload 2

from preparePlots import *
from utils import human_format

In [ ]:
def plotNumOfQuotes():
    
    #read df
    df = plotNumOfQuotesDF()

    #horizontal barplot
    ax = df.plot.barh(stacked=True, figsize=(12, 12), colormap=ListedColormap(sns.color_palette("Set1", n_colors=df.shape[1], desat=.5)))

    #Labels
    ax.set(ylabel="Topic of Article", xlabel="Percentage of Quotes per Quotee type")
    ax.legend(ncol=1, frameon=True)
    plt.legend(bbox_to_anchor=(1.03, 1), loc=2, borderaxespad=0.)
    locs, labels = plt.xticks()
    labels = [str(int(l*100))+'%' for l in locs]
    plt.xticks(locs, labels)
    plt.xlim(xmin = 0, xmax = 1)

    #Depiction
    plt.savefig('plots/CountQuotes.png', bbox_inches = 'tight', transparent=True)
    plt.show()
plotNumOfQuotes()

In [ ]:
def plotHeatMap():
    
    #read df
    df = plotHeatMapDF()
    
    #Aesthetics
    sns.set(style="whitegrid")
    f, ax = plt.subplots(figsize=(12, 12))
    #sns.set_color_codes("deep")

    #Plot
    ax = sns.heatmap(df, cmap="Blues", vmax=0.5)

    #Labels
    ax.set(ylabel="Topic of Article", xlabel="Topic of Quote")
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([l/100 for l in [0, 25, 50]])
    cbar.set_ticklabels([str(l)+'%' for l in [0, 25]] + ['>50%'])
    
    #Depiction
    plt.savefig('plots/TopicsHeatMap.png', bbox_inches = 'tight', transparent=True)
    plt.show()    
plotHeatMap()

In [ ]:
def plotTopQuotees(n=20):
    
    #read df
    df_p, df_o = plotTopQuoteesDF()
    df_p = df_p.head(n)
    df_o = df_o.head(n)

    #Aesthetics
    sns.set(style="whitegrid")
    f, ax = plt.subplots(figsize=(12, 12))

    #Barplots
    sns.set_color_codes("deep")
    sns.barplot(x="count", y="quotee", data=df_p, ax=ax, label="total", color='r')

    #Labels
    ax.set(ylabel="Quotee Name", xlabel="Number of Quotes")
    sns.despine()
    locs, labels = plt.xticks()
    labels = [human_format(l) for l in locs]
    plt.xticks(locs, labels)

    #Depiction
    plt.savefig('plots/TopQuotees.png', bbox_inches = 'tight', transparent=True)
    plt.show()
     

    #Aesthetics
    sns.set(style="whitegrid")
    f, ax = plt.subplots(figsize=(12, 12))

    #Barplots
    sns.set_color_codes("deep")
    sns.barplot(x="count", y="organization", data=df_o, ax=ax, label="total", color='r')

    #Labels
    ax.set(ylabel="Organization Name", xlabel="Number of Quotes")
    sns.despine()
    locs, labels = plt.xticks()
    labels = [human_format(l) for l in locs]
    plt.xticks(locs, labels)

    #Depiction
    plt.savefig('plots/TopOrganizations.png', bbox_inches = 'tight', transparent=True)
    plt.show()
    
plotTopQuotees()